In [1]:
from __future__ import annotations

%load_ext autoreload
%autoreload 2
import math
import sys
from pathlib import Path

sys.path.append('/media/data/robert/code/bids')
from bids_files import BIDS_FILE
import numpy as np
from nii_utils import calc_centroids, centroids_to_dict
import nibabel as nib

subreg_nib: nib.Nifti1Image = nib.load("vertebra_align/subreg.nii.gz")
vert_nib = nib.load("vertebra_align/vert.nii.gz")
subreg_arr1 = subreg_nib.get_fdata()
vert_arr1 = vert_nib.get_fdata()
arr = np.zeros_like(subreg_arr1)

FileNotFoundError: No such file or no access: 'vertebra_align/subreg.nii.gz'

In [152]:
vert_arr_dict:dict[int,np.ndarray] = {}
for vert_id in np.unique(vert_arr1):
    # vert_id = 17
    print(vert_id)
    subreg_arr = subreg_arr1.copy()
    vert_arr = vert_arr1.copy()
    vert_arr[vert_arr != vert_id] = 0
    vert_arr[vert_arr == vert_id] = 1
    subreg_arr *= vert_arr
    vert_arr_dict[vert_id] = subreg_arr

0.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0


In [153]:
import matplotlib.pyplot as plt
import numpy as np


import open3d as o3d

#o3d_bbox.R
#np.asarray(o3d_bbox.get_box_points())


In [159]:
arr = np.zeros_like(subreg_arr1)
import numpy as np
import SimpleITK as sitk
for vert_id in np.unique(vert_arr1)[1:]:
    x = vert_arr_dict[vert_id]
    inp = np.array(np.where(x!=49))
    if inp.shape[-1] == 0:
        continue
    
    sub_pcd_o3d = o3d.utility.Vector3dVector(inp.T)
    o3d_bbox = o3d.geometry.OrientedBoundingBox.create_from_points(sub_pcd_o3d)
    extent = o3d_bbox.extent
    image_2D = sitk.Image(arr.shape, sitk.sitkFloat32)
    image_2D*=0
    extent = [int(round(i)) for i in extent]
    
    image_2D[:int(extent[-1]),:int(extent[-2]),:int(extent[-3])] = 1
    img = sitk.GetImageFromArray(arr)
    
    translation = sitk.TranslationTransform(3, list(reversed(o3d_bbox.center)))
    translation_extend = sitk.TranslationTransform(3, [i/2 for i in reversed(o3d_bbox.extent)])
    rotation3 = sitk.VersorTransform()
    rotation3.SetCenter([-i for i in reversed(o3d_bbox.center)])
    rot = np.array(o3d_bbox.R[::-1,::-1])#
    rotation3.SetMatrix(rot.reshape(-1))

    composite = sitk.CompositeTransform(translation_extend)
    composite.AddTransform(rotation3.GetInverse())
    composite.AddTransform(translation.GetInverse())

    image_2D = sitk.Resample(
            image_2D,
            img,
            composite,
            sitk.sitkLinear,
            0.0,
            image_2D.GetPixelID(),
        )
    out = sitk.GetArrayFromImage(sitk.Cast(image_2D, sitk.sitkInt32))
    arr[out!=0] = vert_id




In [160]:
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html


nib.save(
    nib.Nifti1Image(arr, subreg_nib.affine),
    "vertebra_align/centroids2.nii.gz",
)

In [156]:
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html


nib.save(
    nib.Nifti1Image(arr, subreg_nib.affine),
    "vertebra_align/centroids2.nii.gz",
)